In [1]:
import numpy as np
import matplotlib.pyplot as plt

from one.api import ONE
import brainbox.behavior.wheel as wh
from brainbox.processing import *

one = ONE(base_url='https://alyx.internationalbrainlab.org')

Downloading: /Users/kainylund/Downloads/ONE/alyx.internationalbrainlab.org/tmpts1a8x04/cache.zip Bytes: 80403633


877321393it [00:15, 57702838.10it/s]                             


In [4]:
def plot_event_avgs(clu_event_avgs, avg_event_idxs):
  avgs = clu_event_avgs["avgs"]
  clusters = clu_event_avgs["clusters"]
  for event in avg_event_idxs:
    plt.axvline(x=event, color="gray")
  for i in range(len(avgs[0])):
    plt.plot(avgs[0][i])
  plt.ylim(bottom=-1, top=50)
  plt.title("Event averaged firing rates for " + str(len(clusters)) + " clusters in pid " + pid)
  plt.show()

In [ ]:
SCALED_LEN = 250 # Number of values to interpolate spike rates into
EID_INFO_PATH = "./brainwide_eid_info.npy"  # Path to eid info dictionary

eid = "0802ced5-33a3-405e-8336-b65ebc5cb07c"
pid = "7d999a68-0215-4e45-8e6c-879c6ca2b771"

# Load all trial data
all_trials, left_corr, left_inc, right_corr, right_inc = event_timing_by_trial_type(eid)

# Precomputed avg value indices for stimon, first wheel move, and feedback when SCALED_LEN = 250
avg_event_lengths = [37, 51, 62]  # UNITS: values
avg_event_idxs = list(np.cumsum(avg_event_lengths)) # [37, 88, 150]
event_names = ["stimOn_times", "first_wheel_move", "feedback_times"]

# Since one of our events is first_wheel_move, append the correponding times to the
# trial timing dataframes
wheel = one.load_object(eid, 'wheel', collection='alf', attribute=['position', 'timestamps'])
pos, t = wh.interpolate_position(wheel.timestamps, wheel.position, freq=1000)
all_trials = append_session_wheel_movements(t, pos, all_trials)
left_corr = append_session_wheel_movements(t, pos, left_corr)
left_inc = append_session_wheel_movements(t, pos, left_inc)
right_corr = append_session_wheel_movements(t, pos, right_corr)
right_inc = append_session_wheel_movements(t, pos, right_inc)
trial_timing_dfs = [all_trials, left_corr, left_inc]

# Toy example for a single session at the repeated site
clu_event_avgs = event_average_session_firing_rates(pid, trial_timing_dfs, event_names,
                                                avg_event_idxs, scaled_len=SCALED_LEN)
plot_event_avgs(clu_event_avgs, avg_event_idxs)